__No te olvides de rellenar esto:__

- Número de grupo:
- Nombre de los integrantes del grupo:

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de lcases mediantes las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q695269' -> {'Q25630013', 'Q3388256', 'Q524526', 'Q846109', 'Q960389'} )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q34379' -> {'Q1879241', 'Q54820129'} )

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

```python
import json

with open(filename) as f:
    data = json.load(f)
```

In [238]:
import json

In [548]:
class Taxonomy:
    
    """Inicialización de la taxonomía"""
    def __init__(self):
        #Diccionarios
        self.dictionaryIdLabel = dict() #Par clave (id) - valor (label)
        self.dictionaryClassSubclass = dict() #Par clave (id) - valor (lista de id de subclases)
        self.dictionaryClassSuperclass = dict() #Par clave (id) - valor (lista de id de superclases)
        
        #Variables necesarias para la operacion path
        self.minPath = list()
        self.minDistance = 0
        
        
    """Carga del grafo en memoria""" 
    def load(self, jsonFile):
        with open(jsonFile,encoding='utf-8') as f:
            data = json.load(f)
            
        for d in data:
            
            #Coger etiqueta de X
            labelX = d.get('xLabel')
            #Coger identificador de X
            idX = d.get('x')
            idX = idX.replace('http://www.wikidata.org/entity/', '')
            #Coger etiqueta de Y
            labelY = d.get('yLabel')
            #Coger identificador de Y
            idY = d.get('y')
            idY = idY.replace('http://www.wikidata.org/entity/', '')

            
            #Cargar X e Y en el diccionario que asocia cada identificador con su etiqueta
            self.dictionaryIdLabel[idX] = labelX
            self.dictionaryIdLabel[idY] = labelY
                
            #Cargar X como subclase de Y
            
            if self.dictionaryClassSubclass.get(idY) == None:
                self.dictionaryClassSubclass[idY] = list()
                
            listaY = self.dictionaryClassSubclass.get(idY)
            if idX not in listaY:
                listaY.append(idX)
        
            #Cargar superclase de X
            if self.dictionaryClassSuperclass.get(idX) == None:
                self.dictionaryClassSuperclass[idX] = list()
                
            listaS = self.dictionaryClassSuperclass.get(idX)
            if idY not in listaS:
                listaS.append(idY)
            
    """Mostrar por pantalla la jerarquía de la taxonomia desde una determinada clase
    y hasta un determinado nivel de profundidad"""     
    def print_tree(self, clase, i, numNiveles):
        
        s = ""
        for j in range(i):
            s += "  " 
        print(s, i, self.dictionaryIdLabel[clase], '('+clase+')')
        
        if (i < numNiveles):
            for instrument in self.dictionaryClassSubclass[clase]:
                #Si la subclase tiene mas subclases se hace llamada recursiva
                #Si no se imprime subclase
                if (instrument in self.dictionaryClassSubclass): 
                    self.print_tree(instrument, i + 1, numNiveles)
                else:
                    s = ""
                    for j in range(i + 1):
                        s += "  "
                    print(s, i + 1, self.dictionaryIdLabel[instrument], '('+instrument+')')
        
        
        
    def LCSmio(self, A, B):
        
        sol=[]
        
        #Caso base: A == B
        if A == B:
            sol.append(A)
            return sol
        else:
            #Coger superclases de A y comprobar si estan en B 
            for sA in self.dictionaryClassSuperclass[A]:
                if sA in self.dictionaryClassSuperclass[B]: #Si coinciden se añade a la solucion
                    sol.append(sA)
                else: #Si no coincide buscar recursivamente en las superclases de sA
                    aux = self.dictionaryClassSuperclass[sA];
                    if not aux:
                        aux.extend(LCS_rec(aux,B,sol))
                        
            #Coger superclases de B y comprobar si estan en A
            for sB in self.dictionaryClassSuperclass[B]:
                #Buscar recursivamente en las superclases de sB
                aux = self.dictionaryClassSuperclass[sB];
                if not aux:
                    aux.extend(LCS_rec(aux,A,sol))
            
        return sol
    
    def LCS_rec(self, C1, C2, sol):
        #Coger superclases de C1 y comprobar si estan en C2
        for c in self.dictionaryClassSuperclass[C1]:
            if c in self.dictionaryClassSuperclass[C2]: #Si coinciden se añade a la solucion
                sol.append(c)
            else:
                LCS_rec(c,C2,sol)
                
        return sol
    
    """Función que obtiene todas las superclases de una clase"""
    def getAllSuperClasses(self, clase, listSuperClasses):
        
        if (clase in self.dictionaryClassSuperclass):
            for instrument in self.dictionaryClassSuperclass[clase]:
                listSuperClasses.append(instrument)
                self.getAllSuperClasses(instrument, listSuperClasses)
    
    
    """Función que obtiene todas las subclases de una clase"""
    def getAllSubClasses(self, clase, listSubClasses):
        
        if (clase in self.dictionaryClassSubclass):
            for instrument in self.dictionaryClassSubclass[clase]:
                listSubClasses.append(instrument)
                self.getAllSubClasses(instrument, listSubClasses)
                
    
    """Least Common Subsummer"""
    def LCS(self, classA, classB):
        
        listSuperClassesA = list()
        listSuperClassesA.append(classA)
        self.getAllSuperClasses(classA, listSuperClassesA)
        listSuperClassesB = list()
        listSuperClassesB.append(classB)
        self.getAllSuperClasses(classB, listSuperClassesB)
        
        sol = list()
        for instrument in listSuperClassesA:
            if (instrument in listSuperClassesB):
                if (len(sol) > 0):
                    if (self.verify(instrument, sol)):
                        sol.append(instrument)
                else:
                    sol.append(instrument)
        
        return (sol)
    
    """Función que verifica que una clase no sea más
    Específica que otra"""
    def verify(self, instrument, solution):
        
        listSubClassesA = list()
        listSubClassesA.append(instrument)
        self.getAllSubClasses(instrument, listSubClassesA)
        
        encontrado = False
        for sol in solution:
            if sol in listSubClassesA:
                encontrado = True
        
        return (not encontrado)
    
    
    """Función que encuentra el camino mínimo entre A y B"""
    def path(self, A, B):
        sol = list()
        self.minDistance = 0
        self.minPath = list()
        self.minPath.append(A)
        
        if(A != B): #Encontrar camino hasta B desde A
            path = self.minPath.copy()
            distance = 1
            self.find_path(A, B, distance, path) 
            
        for i in self.minPath:
            sol.append(self.dictionaryIdLabel[i]+' ('+i+')')
       
        return sol
                

    """Función recursiva auxiliar para encontrar el camino mínimo"""            
    def find_path(self, A, B, distance, path):
        
        if A in self.dictionaryClassSubclass:
            for i in self.dictionaryClassSubclass[A]: #Recorrido en anchura
                path.append(i) 
                
                #Si encontramos a B devolvemos el camino con la distancia
                #Si todavia no encontramos a B seguimos buscando en profundidad
                if i == B: 
                    if self.minDistance == 0 or distance < self.minDistance:
                        self.minPath = path.copy()
                        self.minDistance = distance

                else:  #Recorrido en profundidad
                    self.find_path(i, B, distance+1, path)
                    
                path.remove(i)
                
    """Función que calcula la similitud entre dos conceptos A y B.
    Dicha similitud se calcula con la fórmula al inicio de este documento."""   
    def similarity(self, A, B):
        C = self.LCS(A,B)
        root = 'Q34379'
        for i in C:
            xPath = self.path(root,i) #Camino minimo entre la raiz y C
            x = self.minDistance
            yPath = self.path(i,A)    #Camino minimo entre C y A
            y = self.minDistance
            zPath = self.path(i,B)    #Camino minimo entre C y B
            z = self.minDistance

        
        #Aplicar formula para obtener la similitud entre A y B
        sim = x / (x + y + z)
        
        #Descomentar para mostrar los caminos minimos
        #print(xPath,yPath,zPath)
        
        return sim
        
        

In [549]:
#Crear la taxonomia
t = Taxonomy()

In [550]:
#Cargar datos de la consulta 1
t.load('query.json')

### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [554]:
t.print_tree('Q17172850', 0, 2)

 0 voice (Q17172850)
   1 bass (Q27911)
     2 bass-baritone (Q810480)
     2 buffo bass (Q1002146)
     2 basso profondo (Q2532487)
     2 lyric high bass (Q3636053)
     2 Q5885030 (Q5885030)
     2 octavist (Q7082656)
     2 character bass (Q20638448)
     2 high bass (Q54636007)
     2 dramatic high bass (Q54636036)
     2 serious bass (Q54636068)
     2 heavy acting bass (Q54636271)
     2 Bass bourdon (Q64363543)
   1 baritone (Q31687)
     2 bass-baritone (Q810480)
     2 character baritone (Q1062931)
     2 heldenbaritone (Q1601737)
     2 buffo baritone (Q5721499)
     2 light baritone (Q5721503)
     2 Q5885030 (Q5885030)
     2 Lyric baritone (Q8243255)
     2 dramatic baritone (Q8243257)
     2 baryton-noble (Q19740895)
     2 baryton-Martin (Q21478751)
     2 Q25404193 (Q25404193)
     2 Verdi baritone (Q54635681)
     2 acting baritone (Q54635751)
     2 Kavalierbariton (Q54635784)
   1 contralto (Q37137)
     2 acting alto (Q5785182)
     2 dramatic contralto (Q5785183)


### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
```

In [555]:
t.LCS('Q6012297','Q54634726')

['Q186506']

In [556]:
t.LCS('Q5885030','Q6012300')

['Q17172850']

In [557]:
t.LCS('Q27914','Q27914')

['Q27914']

In [558]:
t.LCS('Q15336282','Q64166304')

['Q105738', 'Q1798603', 'Q55724840']

### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profunidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q55724840') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

In [559]:
t.path('Q186506', 'Q54634726')

['mezzo-soprano (Q186506)', 'light mezzo-soprano (Q54634726)']

In [560]:
t.path('Q17172850', 'Q6012300')

['voice (Q17172850)', 'alto (Q6983813)', 'Q6012300 (Q6012300)']

In [561]:
t.path('Q27914', 'Q27914')

['tenor (Q27914)']

In [562]:
t.path('Q34379', 'Q55724840')

['musical instrument (Q34379)',
 'chordophone (Q1051772)',
 'composite chordophones (Q19588495)',
 'lutes (Q1808578)',
 'handle lutes (Q30038759)',
 'necked lutes (Q55724833)',
 'necked box lutes (Q55724840)']

### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [563]:
t.similarity('Q6012297', 'Q54634726')

0.5

In [564]:
t.similarity('Q186506', 'Q54634726')

0.6666666666666666

In [565]:
t.similarity('Q27914', 'Q27914')

1.0

In [566]:
t.similarity('Q76239', 'Q78987')

0.42857142857142855

### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

In [567]:
#Piano-Guitarra (PG)
t.similarity('Q5994','Q6607')

0.2727272727272727

In [568]:
#Piano-Guitarra electrica (PGE)
t.similarity('Q5994','Q78987')

0.25

In [569]:
#Piano-flauta (PF)
t.similarity('Q5994','Q11405')

0.0

In [570]:
#Piano-Trompeta (PT)
t.similarity('Q5994','Q8338')

0.0

In [571]:
#Guitarra-Guitarra electrica (GGE)
t.similarity('Q6607','Q78987')

0.8333333333333334

In [572]:
#Guitarra-Flauta (GF)
t.similarity('Q6607','Q11405')

0.0

In [573]:
#Guitarra-Trompeta (GT)
t.similarity('Q6607','Q8338')

0.0

In [574]:
#Guitarra electrica-Flauta (GEF)
t.similarity('Q78987','Q11405')

0.0

In [575]:
#Guitarra electrica-Trompeta (GET)
t.similarity('Q78987','Q8338')

0.0

In [576]:
#Flauta-Trompeta (FT)
t.similarity('Q11405','Q8338')

0.4

#### Resultados

<table>
  <thead>
    <tr>
      <th>PG</th>
      <th>PGE</th>
      <th>PF</th>
      <th>PT</th>
      <th>GGE</th>
      <th>GF</th>
      <th>GT</th>
      <th>GEF</th>
      <th>GET</th>
      <th>FT</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>0.2727272727272727</td>
      <td>0.25</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.8333333333333334</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.4</td>
    </tr>
  </tbody>
</table>


#### Comentarios